In [45]:
import qgis
from qgis.core import QgsVectorLayer

import logging as log
log.getLogger().setLevel(log.INFO)

In [46]:
import sqlite3
from time import strftime, time
# now = datetime.now()

# strftime('%Y-%m-%dT%H:%M:%fZ',now)

In [47]:
gpkg = "/data/SciDataHub/projects/planmap_maps/crommelin/Crommelin_draft_v2.gpkg"
con = sqlite3.connect(gpkg)
cc = con.cursor()

In [48]:

# cc.execute('CREATE TABLE IF NOT EXISTS "layer_styles" \
#     ( "id" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, \
#     "f_table_catalog" TEXT(256), \
#            "f_table_schema" TEXT(256), \
#            "f_table_name" TEXT(256), \
#            "f_geometry_column" TEXT(256), \
#            "styleName" TEXT(30), \
#            "styleQML" TEXT, "styleSLD" TEXT, "useAsDefault" BOOLEAN, "description" TEXT, \
#            "owner" TEXT(30), "ui" TEXT(30), \
#            "update_time" DATETIME DEFAULT (strftime('%Y-%m-%dT%H:%M:%fZ','now')))')


# cc.execute('CREATE TRIGGER  IF NOT EXISTS "trigger_insert_feature_count_layer_styles" AFTER INSERT ON "layer_styles" BEGIN UPDATE gpkg_ogr_contents SET feature_count = feature_count + 1 WHERE lower(table_name) = lower('layer_styles'); END')

In [49]:
# c.execute("INSERT INTO {tn} ({idf}, {cn}) VALUES (123456, 'test')".\
#         format(tn=table_name, idf=id_column, cn=column_name))

In [50]:
from osgeo import ogr
import os
sfilefolder = "/data/SciDataHub/projects/planmap_maps/crommelin/Crommelin_Styles/"
conn = ogr.Open(gpkg)

for i in conn:
    # append the layername part
    name  = i.GetName()
    if name == "layer_styles":
        continue
    
    # match the style file 
#     log.info(name)
    
    tfile = os.path.join(sfilefolder, name.strip() + ".qml")
    tfile2 = os.path.join(sfilefolder, name.strip() + ".sld")
#     log.info(f"trying {tfile}")
    
    
    if not os.path.exists(tfile):
        continue
    
    log.info(f"matched {tfile}")
        
    with open(tfile, 'r', encoding='utf-8') as file:
        data_qml = file.read() 
        
        
    with open(tfile2, 'r', encoding='utf-8') as file:
        data_sld = file.read() 
        
#     data_qml = data_qml.encode(encoding=None)
#     data_sld = data_sld.encode(encoding=None)
        
#     q = f"INSERT INTO layer_styles (f_table_name, f_geometry_column, styleName, styleQML, styleSLD, useAsDefault) VALUES (\'{name}\', 'geom', \'{name}\', \'{data_qml}\', '{data_sld}', '1');"
    q = f"INSERT INTO layer_styles (description, f_table_name, f_geometry_column, styleName, useAsDefault, styleQML, styleSLD) VALUES ('Fri May 10 16:35:07 2019', '{name}', 'geom', '{name.strip()}', 1, ?, ?);"

#     log.info(f"query {q}")
    cc.execute(q, [data_qml, data_sld])
        
#     cursor.execute("Update data SET raw_str=? where id=1", [long_string])
    
#     log.info(name)
    
#     fullpath = gpkg + f"|layername={name}"
#     log.info(f"fullpath {fullpath}")
    
#     vlayer = QgsVectorLayer(fullpath, name, "ogr")
    
#     if not vlayer.isValid():
#         print("Layer failed to load!")
    


INFO:root:matched /data/SciDataHub/projects/planmap_maps/crommelin/Crommelin_Styles/Aeolian_Deposits.qml
INFO:root:matched /data/SciDataHub/projects/planmap_maps/crommelin/Crommelin_Styles/Surrounding_terrains.qml
INFO:root:matched /data/SciDataHub/projects/planmap_maps/crommelin/Crommelin_Styles/Units_Central_Crater_Bulge.qml
INFO:root:matched /data/SciDataHub/projects/planmap_maps/crommelin/Crommelin_Styles/Units_Crater_Rim.qml
INFO:root:matched /data/SciDataHub/projects/planmap_maps/crommelin/Crommelin_Styles/Units_Inner_Crater_Layered.qml
INFO:root:matched /data/SciDataHub/projects/planmap_maps/crommelin/Crommelin_Styles/Units_Secondary_Craters_Related.qml


In [51]:
con.commit()
con.close()